# 2021-22 Corporate Tax Transparency Geospatial Analysis

Analysis of the ATO's 2021-22 Corporate Tax Transparency Report.

Producing these visuals required the use of Postcodes gathered from the ABR's ABN Bulk Extract, and linking these postcodes to the ABS's ASGS Geospatial Datasets (specifically, LGA).

*Transparency Report entities have been filtered to only those with ABNs, and postcodes as available in the ABR.*

## Relevant Imports

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import pathlib

## Read-In & Form Corporate Tax Transparency with Geocoding

### 2021-22 Corporate Tax Transparency Data, with Postcodes

In [2]:
transparency_data_df = pd.read_csv(pathlib.Path.cwd() / 'main_data' / '2021-22_corporate_tax_transparency_postcode_data.csv', dtype={'abn': str, 'postcode': str})

transparency_data_df

,name,abn,total_income,taxable_income,tax_payable,income_year,postcode
0,1884 PTY LIMITED,83114980880,215297043,8124632.0,2367120.0,2021-22,2007
1,20 CASHEWS PTY LTD,16634403124,288582715,36977225.0,962761.0,2021-22,3000
2,21ST CENTURY RESORTS HOLDINGS PTY LIMITED,33104201014,255202722,19673962.0,5902189.0,2021-22,2065
3,29METALS LIMITED,95650096094,501817901,NaN,NaN,2021-22,3000
4,2SAN PTY LTD,93641633141,739184508,70223411.0,21067023.0,2021-22,2000
...,...,...,...,...,...,...,...
2685,ZFAAS HOLDINGS PTY LTD,87609580690,219495534,206108050.0,61832415.0,2021-22,4006
2686,ZIMMER AUSTRALIA HOLDING PTY LIMITED,93107449534,282546149,17665597.0,5299679.0,2021-22,2086
2687,ZIP CO LIMITED,50139546428,304646327,NaN,NaN,2021-22,2000
2688,ZOETIS AUSTRALIA PTY LTD,94156476425,469791424,26261399.0,7878420.0,2021-22,2138


### Geonames Postcodes & Geocoding

In [3]:
geonames_postcodes_gdf = (gpd.read_file(f'zip:///{pathlib.Path.cwd() / 'geo_data' / 'au_postcodes.zip'}')
                          .assign(postal_cod = lambda x: x['postal_cod'].astype('str')))

geonames_postcodes_gdf

,postal_cod,place_name,geometry
0,200,Australian National University,POINT (149.11890 -35.27770)
1,221,Barton,POINT (149.14120 -35.30490)
2,2540,Jervis Bay,POINT (150.69690 -35.14990)
3,2540,Hmas Creswell,POINT (150.55010 -35.02800)
4,2540,Wreck Bay,POINT (150.69080 -35.16480)
...,...,...,...
16867,6989,Maddington,POINT (115.98330 -32.05000)
16868,6990,Gosnells,POINT (116.00540 -32.08100)
16869,6991,Kelmscott,POINT (116.02590 -32.12430)
16870,6992,Armadale,POINT (116.00930 -32.14610)


In [4]:
transparency_data_postcodes_gdf = (geonames_postcodes_gdf
                                   .merge(transparency_data_df, how='right', left_on='postal_cod', right_on='postcode')
                                   [['name', 'abn', 'total_income', 'taxable_income' ,'tax_payable', 'geometry']]
                                   .query('not(geometry.isna())')
                                   .drop_duplicates(ignore_index=True)
                                   .to_crs(7844)
                                  )

del geonames_postcodes_gdf, transparency_data_df

transparency_data_postcodes_gdf

,name,abn,total_income,taxable_income,tax_payable,geometry
0,1884 PTY LIMITED,83114980880,215297043,8124632.0,2367120.0,POINT (151.20250 -33.86990)
1,1884 PTY LIMITED,83114980880,215297043,8124632.0,2367120.0,POINT (151.19770 -33.88060)
2,20 CASHEWS PTY LTD,16634403124,288582715,36977225.0,962761.0,POINT (144.96110 -37.81300)
3,21ST CENTURY RESORTS HOLDINGS PTY LIMITED,33104201014,255202722,19673962.0,5902189.0,POINT (151.20060 -33.82210)
4,21ST CENTURY RESORTS HOLDINGS PTY LIMITED,33104201014,255202722,19673962.0,5902189.0,POINT (151.18240 -33.83400)
...,...,...,...,...,...,...
14743,ZURICH FINANCIAL SERVICES AUSTRALIA LIMITED,11008423372,4965391466,117579598.0,11750915.0,POINT (151.19620 -33.82200)
14744,ZURICH FINANCIAL SERVICES AUSTRALIA LIMITED,11008423372,4965391466,117579598.0,11750915.0,POINT (151.19880 -33.83740)
14745,ZURICH FINANCIAL SERVICES AUSTRALIA LIMITED,11008423372,4965391466,117579598.0,11750915.0,POINT (151.20830 -33.83760)
14746,ZURICH FINANCIAL SERVICES AUSTRALIA LIMITED,11008423372,4965391466,117579598.0,11750915.0,POINT (151.20710 -33.84350)


## ASGS Transparency Data Merge

In [5]:
def asgs_transparency_merge(transparency_gdf: gpd.GeoDataFrame, asgs_name: str, path: pathlib.PosixPath):

    asgs_gdf = (gpd.read_file(f'zip:///{path}')
                .to_crs(7844)
               )

    asgs_gdf[f'{asgs_name}_geometry'] = asgs_gdf['geometry']
    
    initial_merge_gdf = (transparency_gdf
                         .sjoin(asgs_gdf, how="left", predicate="within")
                        )
    
    initial_merge_success_gdf = (initial_merge_gdf
                                 .query('not(index_right.isna())')
                                )
    
    initial_merge_nearest_gdf = (initial_merge_gdf
                                 .query('index_right.isna()')
                                 [['name', 'abn', 'total_income', 'taxable_income', 'tax_payable', 'geometry']]
                                 .sjoin_nearest(asgs_gdf, how="left")
                                )
    
    merge_gdf = (pd.concat([initial_merge_success_gdf, initial_merge_nearest_gdf])
                 [['name', 'abn', 'total_income', 'taxable_income', 'tax_payable', asgs_gdf.columns[1], asgs_gdf.columns[-1]]]
                 .rename(columns={asgs_gdf.columns[1]: f'{asgs_name}_name', f'{asgs_name}_geometry': 'geometry'})
                 .drop_duplicates(ignore_index=True)
                )
    return merge_gdf

## LGA Plots

### Setup

In [6]:
transparency_lga_gdf = asgs_transparency_merge(transparency_data_postcodes_gdf, 'lga', pathlib.Path.cwd() / 'geo_data' / 'lga_2024_aust_gda2020.zip')

transparency_lga_gdf

/home/hobbymath42/miniconda3/envs/corporate_tax_transparency_analysis/lib/python3.12/site-packages/geopandas/array.py:365: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,name,abn,total_income,taxable_income,tax_payable,lga_name,geometry
0,1884 PTY LIMITED,83114980880,215297043,8124632.0,2367120.0,Sydney,"POLYGON ((151.22328 -33.86663, 151.22395 -33.8..."
1,20 CASHEWS PTY LTD,16634403124,288582715,36977225.0,962761.0,Melbourne,"POLYGON ((144.96736 -37.83103, 144.96585 -37.8..."
2,21ST CENTURY RESORTS HOLDINGS PTY LIMITED,33104201014,255202722,19673962.0,5902189.0,North Sydney,"POLYGON ((151.22765 -33.81950, 151.22767 -33.8..."
3,21ST CENTURY RESORTS HOLDINGS PTY LIMITED,33104201014,255202722,19673962.0,5902189.0,Willoughby,"POLYGON ((151.14389 -33.80083, 151.14399 -33.8..."
4,29METALS LIMITED,95650096094,501817901,NaN,NaN,Melbourne,"POLYGON ((144.96736 -37.83103, 144.96585 -37.8..."
...,...,...,...,...,...,...,...
3405,TOPCO INVESTMENTS AUSTRALIA PTY LIMITED,78622116992,672192866,NaN,NaN,Lane Cove,"POLYGON ((151.14524 -33.81585, 151.14527 -33.8..."
3406,VERIZON AUSTRALIA PTY LIMITED,62081001194,185424519,NaN,NaN,Lane Cove,"POLYGON ((151.14524 -33.81585, 151.14527 -33.8..."
3407,VERTEX PHARMACEUTICALS (AUSTRALIA) PTY LTD,34160157157,166449961,NaN,NaN,Lane Cove,"POLYGON ((151.14524 -33.81585, 151.14527 -33.8..."
3408,VIRTUS HEALTH LIMITED,80129643492,260180516,32782023.0,7831063.0,Lane Cove,"POLYGON ((151.14524 -33.81585, 151.14527 -33.8..."


### Total Income

In [7]:
transparency_lga_total_income_gdf = (transparency_lga_gdf
                                     [['lga_name', 'geometry', 'total_income']]
                                     .dissolve(by='lga_name', aggfunc={'total_income': 'sum'})
                                     .assign(total_income_bill=lambda x:x['total_income'] / 1000000000)
                                     .reset_index()
                                     [['lga_name', 'geometry', 'total_income_bill']]
                                    )
transparency_lga_total_income_gdf

,lga_name,geometry,total_income_bill
0,Adelaide,"POLYGON ((138.58014 -34.90598, 138.58064 -34.9...",27.753850
1,Adelaide Plains,"POLYGON ((138.28492 -34.48901, 138.28487 -34.4...",0.584292
2,Albury,"POLYGON ((146.86566 -36.07292, 146.86512 -36.0...",1.201056
3,Alpine,"POLYGON ((146.67058 -36.56827, 146.67056 -36.5...",0.106901
4,Armadale,"POLYGON ((116.06235 -32.20404, 116.05884 -32.2...",0.188895
...,...,...,...
238,Woollahra,"MULTIPOLYGON (((151.27188 -33.87611, 151.27176...",1.265672
239,Woorabinda,"MULTIPOLYGON (((149.40277 -24.62241, 149.39796...",0.170366
240,Wyndham,"POLYGON ((144.70853 -37.80677, 144.70889 -37.8...",5.611107
241,Yarra,"POLYGON ((144.99899 -37.78540, 144.99952 -37.7...",52.904868


In [8]:
lga_map = folium.Map(location=(-25, 135), zoom_start=5)

lga_name = folium.features.GeoJson(
    transparency_lga_total_income_gdf,
    control=False,
    style_function=lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['lga_name', 'total_income_bill'],
        aliases=['LGA Name: ', 'Total Income ($ Billions): '],
        style="""
            background-color: white;
            color: #333333;
            font-family: arial;
            font-size: 12px;
            padding: 10px;
            """,
        localize=True
    )
)

folium.Choropleth(
    geo_data=transparency_lga_total_income_gdf,
    name="choropleth",
    data=transparency_lga_total_income_gdf,
    columns= ["lga_name","total_income_bill"],
    key_on="feature.properties.lga_name",
    fill_color="PuBuGn",
    fill_opacity=0.7,
    line_opacity=0.1,
    bins=6,
    legend_name="Total Income ($ Billions) by LGA, FY2021-22",
    highlight=True
).add_to(lga_map)

lga_map.add_child(lga_name)
folium.LayerControl().add_to(lga_map)

lga_map.save('total_income_by_lga_2021-22.html')

del lga_map, lga_name, transparency_lga_total_income_gdf

### Taxable Income

In [9]:
transparency_lga_taxable_income_gdf = (transparency_lga_gdf
                                     [['lga_name', 'geometry', 'taxable_income']]
                                     .dissolve(by='lga_name', aggfunc={'taxable_income': 'sum'})
                                     .assign(total_taxable_income_mill=lambda x:x['taxable_income'] / 10000000)
                                     .reset_index()
                                     [['lga_name', 'geometry', 'total_taxable_income_mill']]
                                    )
transparency_lga_taxable_income_gdf

,lga_name,geometry,total_taxable_income_mill
0,Adelaide,"POLYGON ((138.58014 -34.90598, 138.58064 -34.9...",173.775457
1,Adelaide Plains,"POLYGON ((138.28492 -34.48901, 138.28487 -34.4...",1.905520
2,Albury,"POLYGON ((146.86566 -36.07292, 146.86512 -36.0...",9.856767
3,Alpine,"POLYGON ((146.67058 -36.56827, 146.67056 -36.5...",0.465934
4,Armadale,"POLYGON ((116.06235 -32.20404, 116.05884 -32.2...",1.990285
...,...,...,...
238,Woollahra,"MULTIPOLYGON (((151.27188 -33.87611, 151.27176...",9.959999
239,Woorabinda,"MULTIPOLYGON (((149.40277 -24.62241, 149.39796...",5.211086
240,Wyndham,"POLYGON ((144.70853 -37.80677, 144.70889 -37.8...",27.572461
241,Yarra,"POLYGON ((144.99899 -37.78540, 144.99952 -37.7...",281.127993


In [10]:
lga_map = folium.Map(location=(-25, 135), zoom_start=5)

lga_name = folium.features.GeoJson(
    transparency_lga_taxable_income_gdf,
    control=False,
    style_function=lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['lga_name', 'total_taxable_income_mill'],
        aliases=['LGA Name: ', 'Total Taxable Income ($10 Millions): '],
        style="""
            background-color: white;
            color: #333333;
            font-family: arial;
            font-size: 12px;
            padding: 10px;
            """,
        localize=True
    )
)

folium.Choropleth(
    geo_data=transparency_lga_taxable_income_gdf,
    name="choropleth",
    data=transparency_lga_taxable_income_gdf,
    columns= ["lga_name","total_taxable_income_mill"],
    key_on="feature.properties.lga_name",
    fill_color="YlOrBr",
    fill_opacity=0.7,
    line_opacity=0.1,
    bins=6,
    legend_name="Total Taxable Income ($10 Millions) by LGA, FY2021-22",
    highlight=True
).add_to(lga_map)

lga_map.add_child(lga_name)
folium.LayerControl().add_to(lga_map)

lga_map.save('total_taxable_income_by_lga_2021-22.html')

del lga_map, lga_name, transparency_lga_taxable_income_gdf

### Tax Payable

In [11]:
transparency_lga_tax_payable_gdf = (transparency_lga_gdf
                                     [['lga_name', 'geometry', 'tax_payable']]
                                     .dissolve(by='lga_name', aggfunc={'tax_payable': 'sum'})
                                     .assign(total_tax_payable_mill=lambda x:x['tax_payable'] / 10000000)
                                     .reset_index()
                                     [['lga_name', 'geometry', 'total_tax_payable_mill']]
                                    )
transparency_lga_tax_payable_gdf

,lga_name,geometry,total_tax_payable_mill
0,Adelaide,"POLYGON ((138.58014 -34.90598, 138.58064 -34.9...",36.559994
1,Adelaide Plains,"POLYGON ((138.28492 -34.48901, 138.28487 -34.4...",0.514918
2,Albury,"POLYGON ((146.86566 -36.07292, 146.86512 -36.0...",2.776705
3,Alpine,"POLYGON ((146.67058 -36.56827, 146.67056 -36.5...",0.000000
4,Armadale,"POLYGON ((116.06235 -32.20404, 116.05884 -32.2...",0.572337
...,...,...,...
238,Woollahra,"MULTIPOLYGON (((151.27188 -33.87611, 151.27176...",2.987931
239,Woorabinda,"MULTIPOLYGON (((149.40277 -24.62241, 149.39796...",1.563326
240,Wyndham,"POLYGON ((144.70853 -37.80677, 144.70889 -37.8...",8.271683
241,Yarra,"POLYGON ((144.99899 -37.78540, 144.99952 -37.7...",77.693384


In [12]:
lga_map = folium.Map(location=(-25, 135), zoom_start=5)

lga_name = folium.features.GeoJson(
    transparency_lga_tax_payable_gdf,
    control=False,
    style_function=lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['lga_name', 'total_tax_payable_mill'],
        aliases=['LGA Name: ', 'Total Tax Payable ($10 Millions): '],
        style="""
            background-color: white;
            color: #333333;
            font-family: arial;
            font-size: 12px;
            padding: 10px;
            """,
        localize=True
    )
)

folium.Choropleth(
    geo_data=transparency_lga_tax_payable_gdf,
    name="choropleth",
    data=transparency_lga_tax_payable_gdf,
    columns= ["lga_name","total_tax_payable_mill"],
    key_on="feature.properties.lga_name",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.1,
    bins=6,
    legend_name="Total Tax Payable ($10 Millions) by LGA, FY2021-22",
    highlight=True
).add_to(lga_map)

lga_map.add_child(lga_name)
folium.LayerControl().add_to(lga_map)

lga_map.save('total_tax_payable_by_lga_2021-22.html')

del lga_map, lga_name, transparency_lga_tax_payable_gdf